<a href="https://colab.research.google.com/github/xzy2022/GeoSeg/blob/main/visualization/plot_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/optuna/optuna-examples/blob/main/visualization/plot_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizing High-dimensional Parameter Relationships

This notebook demonstrates various visualizations of studies in Optuna.
The hyperparameters of a neural network trained to classify images are optimized and the resulting study is then visualized using these features.

**Note:** If a parameter contains missing values, a trial with missing values is not plotted.

In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install Optuna.
! pip install --quiet optuna

## Preparing the Dataset

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml(name="Fashion-MNIST", version=1)
classes = list(set(mnist.target))

# For demonstrational purpose, only use a subset of the dataset.
n_samples = 4000
data = mnist.data[:n_samples]
target = mnist.target[:n_samples]

x_train, x_valid, y_train, y_valid = train_test_split(data, target)

## Defining the Objective Function

In [3]:
from sklearn.neural_network import MLPClassifier


def objective(trial):

    clf = MLPClassifier(
        hidden_layer_sizes=tuple(
            [trial.suggest_int("n_units_l{}".format(i), 32, 64) for i in range(3)]
        ),
        learning_rate_init=trial.suggest_float("lr_init", 1e-5, 1e-1, log=True),
    )

    for step in range(100):
        clf.partial_fit(x_train, y_train, classes=classes)
        value = clf.score(x_valid, y_valid)

        # Report intermediate objective value.
        trial.report(value, step)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.TrialPruned()

    return value

## Running the Optimization

In [4]:
import optuna

optuna.logging.set_verbosity(
    optuna.logging.WARNING
)  # This verbosity change is just to simplify the notebook output.

study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=15)

## Visualizing the Optimization History

In [5]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

## Visualizing the Learning Curves of the Trials

In [6]:
from optuna.visualization import plot_intermediate_values

plot_intermediate_values(study)

## Visualizing High-dimensional Parameter Relationships

In [7]:
from optuna.visualization import plot_parallel_coordinate

plot_parallel_coordinate(study)

### Selecting Parameters to Visualize

In [8]:
plot_parallel_coordinate(study, params=["lr_init", "n_units_l0"])

## Visualizing Parameter Relationships

In [9]:
from optuna.visualization import plot_contour

plot_contour(study)

### Selecting Parameters to Visualize

In [10]:
plot_contour(study, params=["n_units_l0", "n_units_l1"])

## Visualizing Individual Parameters

In [11]:
from optuna.visualization import plot_slice

plot_slice(study)

### Selecting Parameters to Visualize

In [12]:
plot_slice(study, params=["n_units_l0", "n_units_l1"])

## Visualizing Parameter Importances

In [13]:
from optuna.visualization import plot_param_importances

plot_param_importances(study)